# Google Play Store Reviews Scraping


## Purpose
This script automates the process of scraping reviews from a specific app on the Google Play Store, storing them in a MySQL database, performing sentiment analysis, and identifying pros and cons using topic modeling. Finally, the data is exported to a JSON file for further use.

### Features:
1. Scrape reviews using `google-play-scraper`.
2. Store reviews in a MySQL database.
3. Perform sentiment analysis using `TextBlob`.
4. Identify app pros and cons with topic modeling via `sklearn`.
5. Export the data to a JSON file for easy sharing or analysis.

### Usage:
- Replace `app_id` with the ID of the target app on the Google Play Store.
- Ensure MySQL is set up locally, and credentials match the script.
- Run the script in an environment with the necessary libraries installed.



In [36]:
# Install and import relevant libraries
import requests
from bs4 import BeautifulSoup
!pip install mysql-connector-python
import mysql.connector
import json
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from google_play_scraper import app, reviews as fetch_reviews


In [37]:
# Scrape Reviews from Google Play Store
def scrape_reviews(app_id):
    result, _ = fetch_reviews(
        app_id,
        lang='en',  # Language
        country='us',  # Country
        count=10  # Limit to 10 reviews for simplicity
    )
    return [review['content'] for review in result]

In [39]:
# Create MySQL Database
def create_database():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password=""  # Empty password if not set
    )
    cursor = conn.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS AppReviews")
    conn.close()

In [40]:
# Create Table and Load Data
def load_reviews_to_db(reviews):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",  # Empty password if not set
        database="AppReviews"
    )
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Reviews (
            id INT AUTO_INCREMENT PRIMARY KEY,
            review TEXT
        )
    """)

    for review in reviews:
        cursor.execute("INSERT INTO Reviews (review) VALUES (%s)", (review,))

    conn.commit()
    conn.close()

In [41]:
# Sentiment Analysis
def analyze_sentiments(reviews):
    sentiments = [(review, TextBlob(review).sentiment.polarity) for review in reviews]
    return sentiments

In [42]:
# Identify Pros and Cons using Topic Modeling
def identify_topics(reviews):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(reviews)

    lda = LatentDirichletAllocation(n_components=2, random_state=42)
    lda.fit(X)

    topics = []
    for idx, topic in enumerate(lda.components_):
        topics.append([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-5:]])

    return topics

In [43]:
# Export to JSON
def export_to_json():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="reviews_db"
    )
    cursor = conn.cursor(dictionary=True)
    cursor.execute("SELECT * FROM Reviews")
    rows = cursor.fetchall()
    conn.close()

    with open("reviews.json", "w") as f:
        json.dump(rows, f, indent=4)

In [52]:
# Scrape Reviews from Google Play Store
def scrape_reviews(app_id):
    result, _ = fetch_reviews(  # Correctly call fetch_reviews
        app_id,
        lang='en',  # Language
        country='us',  # Country
        count=10  # Limit to 10 reviews for simplicity
    )
    return [review['content'] for review in result]



In [53]:
scrape_reviews('com.mintfintech.app')

["it's a good app, but this is my first time in this website I'm here upload all the service",
 'best',
 'good',
 'Mintyn is the best',
 '🥰🎇',
 'a very silent 5 star Bank saying and delivering on point and on time',
 '😚',
 "it's owesome",
 'So lovely 🥰🥰🥰🥰',
 "My name is odupitan folashade, ma /sir I ave money in the account but I don't know how to withdraw the money is telling me that I should wait till my they contact me,I don't understand the money is from hope"]